### 字典推导

In [ ]:

dial_code=[(1,'a'),(2,'d'),(3,'b')]

mapping={num: code for num,code in dial_code}
print(mapping)


### 映射拆包

In [ ]:
def dump(**kwargs):
    print(type(kwargs))
    return kwargs

# 在dict类型的变量前面加**表示拆包
mapping1={'a':1,**{'b':2},'c':3}
print(dump(**mapping1))

### 使用|合并映射

In [ ]:
m1={'c':1,'d':2,'a':2}
m2={'c':2,'d':1,'r':3}
# | 创建一个新的映射
print(m1|m2) 


### 标准API
`collections.abc`模块中抽象基类Mapping和MutableMapping描述dict和类型类型的接口

> 自定义映射类型,最好通过collecitons.abc.UserDict和组合模式dict这样更好简单点

可哈希:如果一个对象的哈希码在整个生命周期内永不可变(__hash__),而且可与其他对象比较(__eq__)那这个对象就是可哈希.数值类型以及不可变的扁平对象str和bytes均是可哈希,如果容器类型不可变,而且所包含的对象全是可哈希的,那么容器类型自身也是可哈希的.frozenset对象全是可哈希的,tuple里所有对象均是可哈希的才是可哈希

In [ ]:
import collections.abc

my_dict={}
# isinstance 对象是否满足基类要求
print(isinstance(my_dict,collections.abc.Mapping))
print(isinstance(my_dict,collections.abc.MutableMapping))

# 可哈希
# print(hash(my_dict))
tt=tuple((1,2,3)) # 所有元素都是不可变
print(hash(tt))

ti=(1,2,tt)
print(hash(ti))

# Create a new dictionary with keys from iterable and values set to value.
m3=dict.fromkeys(('c'),'2')
print(m3)
print(help(dict.fromkeys))

### 插入或更新可变的
在通过dict['k']访问字典里数据时,如果不存在就会抛出错误,为了避免错误通过dict.get(k,default)这样的方式来处理

### 自动处理缺失键
1. 把普通的dict换成defaultdict

 > 原理在创建对象时提供一个可调用对象,当__getitem__遇到不存在的键时,在调用对象生成一个默认值

2. 定义dict其他类型的子类,实现__missing__方法
 
 > 映射处理缺失键的逻辑在__missing__方法,dict基类本身没有定义这个方法,但是如果dict子类定义了这个方法,那么点那个dict.__getitem__找不到键时在调用__missing__方法,不抛出KeyError

In [2]:
d={}
# d['key']
d.get('k',1)

# defaultdict
from collections import defaultdict


d=defaultdict(lambda :1)
d['2']
print(help(defaultdict))

class StrDict(dict):
    
    def __missing__(self,key):
        # 如果不做判断,self[key]会调用__getitem__再次进入到该方法
        # 防止递归
        if isinstance(key,str):
            raise KeyError(key)
        return self[str(key)]

    def new_method(self, key):
        return key
        

    def get(self,key,default=None):
        try:
            return self[key]
        except(KeyError):
            return default
    
    def __contains__(self,key):
        return key in self.keys() or str(key) in self.keys() 

my=StrDict()
my['2']=2
my[2]

Help on class defaultdict in module collections:

class defaultdict(builtins.dict)
 |  defaultdict(default_factory=None, /, [...]) --> dict with default factory
 |  
 |  The default factory is called without arguments to produce
 |  a new value when a key is not present, in __getitem__ only.
 |  A defaultdict compares equal to a dict with the same items.
 |  All remaining arguments are treated the same as if they were
 |  passed to the dict constructor, including keyword arguments.
 |  
 |  Method resolution order:
 |      defaultdict
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __copy__(...)
 |      D.copy() -> a shallow copy of D.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __missing__(...)
 |      __missing__(key) # Called by __getitem__ for missing key; pseudo-code:
 |      if self.defau

2

### dict的变体
- collections.OrderedDict
    - 类似于LRU
    - popitem 通过一个可选参数指定移除哪一项
    - move_to_end 把元素位置移到某一端
- collections.ChainMap
    - 用来存放一组映射,可视为整体查找.查找操作按照输入映射在构造函数调用中出现的顺序执行,一旦在某个位置找到指定的键,旋转结束.
- collections.Counter
    - 统计出现个数
- shelve.Shelf
    - shelve模块持久化存储字符串与python对象之间的映射

### 不可变映射
 标准库提供的映射都是不可变的,在types块提供的MappingProxyType类,把传入的映射包装成一个mapppingproxy实例,这原映射的动态代理,只可读取.

### 字典视图
- keys():返回dict_key
- values():返回dict_value
- items():返回dict_key和dict_value

In [13]:

class MyClass:
    _class=2
    def __init__(self,name,type) :
        self.name=name
        self.type=type


my=MyClass('harden','type')

print(my.__dict__)

#不可变映射
from types import MappingProxyType

proxy=MappingProxyType(dict(one=1,two=2))
print(proxy['one'])
print(proxy)
# proxy['one']=2

#  python默认在特殊,特殊的__dict__中存储实例属性,这个属性的值是一个字典附在实例上
print(id(my.__dict__))

{'name': 'harden', 'type': 'type'}
1
{'one': 1, 'two': 2}
4505953536


### 集合
集合元素必须是可哈希对象,set类型不可哈希,因此不能构建嵌套set,但是可以通过frozenset进行构造

- set和frozenset

- 集合推导式:`{i for i in range(10)}`

- set字面量
    > 集合的字面量可以写成a={1,2} b={1},空的字面量必须是set()

    > frozenset没有字面量语句,必须通过构造函数创建

- 集合运算
    1. intersection 交集
    2. union 并集
    3. differece 差集
    4. issubset 判断是否是子集 <=
    5. issuperset 判断是否是超集 >=
    6. dicard(d) 从集合中删除(如果存在)
    7. `&` 交集
    8. `|` 并集
    9. `-` 差集
    10. `^` 对差集

In [5]:
# 计算交集个数
a={i for i in range(4)}
b={i for i in range(2)}
print(len(a&b))
print(len(a.intersection(b)))

c=frozenset(range(10))
print(type(c))

print(a.intersection(b))
print(a.union(b))
print(a.difference(b))

2
2
<class 'frozenset'>
{0, 1}
{0, 1, 2, 3}
{2, 3}
